# Analysing forest recovery after logging events - comparison with satellite data

During the previous practical, you learned about the data that the Victorian Department of Environment, Land, Water and Planning keep on the harvesting of logging events over time, and exported all the events that can be easily compared with satellite data from the Sentinel-2 constellation.

Your team has reviewed your file and selected three known logging events, which they would like you to investigate. Each event has a different type of harvesting strategy, and your team are curious about how these events show up in satellite data. 

In this practical, you'll load and view the satellite data associated with each logging event, both as an image, and using the normalised difference vegetation index (NDVI). You'll then view the time series of how NDVI is changing over time for each event. 

## Overview

During this activity, you will learn to

* select specific rows from a table of geospatial data
* load satellite data for the time and location corresponding to a given logging event
* review satellite data, both as images and as timeseries

All of the Python code for this session has been provided for you. The focus of this session is to review the results and think about what you can learn from satellite data.

## Notebook setup

In addition to `pandas` and `geopandas`, you'll now also need the `datacube` package; this is what lets you load satellite data. The other analyst on your team has also provided a number of extra functions that you'll use throughout the notebook.

To run the code, click on the next cell, and press `Shift`+`Enter` on your keyboard.

In [ ]:
# import key packages
import datacube
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# import extra functions
from datacube.utils.geometry import Geometry
from dea_tools.datahandling import load_ard
from dea_tools.spatial import xr_rasterize
from odc.algo import xr_geomedian
from plotting_functions import plot_rgb_ndvi

# Change a pandas setting to view all columns and all rows of loaded data
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## Load the data

In the last practical, you exported a file called "LOG_SEASON_FILTERED.gpkg" to the "data" folder in your workspace. This contains all the logging events that can be matched with Sentinel-2 data. 

The code in the next cell will load the "LOG_SEASON_FILTERED.gpkg" file, and assign it to a variable called `logging_season_data`

In [ ]:
logging_season_data = gpd.read_file("data/LOG_SEASON_FILTERED.gpkg")

It is a good idea to look at the first five rows of your loaded data to check it has been imported correctly. To do this, you can use the `.head()` function:

In [ ]:
logging_season_data.head()

## Select logging events for analysis

Your team has identified three logging events for you to investigate, with each event demonstrating a different harvesting approach. They have provided you with the event LOGHISTID values so that you can select them from your list of events:

**Clearfelling event**: "14/770/507/0011/201920/00"
**Regrowth retention harvesting event**: "08/286/505/0029/201920/00"
**Variable retention 1 event**: "16/686/510/0026/201920/00"

Your colleague has provided these event keys as a Python dictionary, and has provided some code to only select the rows in your dataset with these LOGHISTID values. Run the cells below, and then complete the exercise.

In [ ]:
events_of_interest = {
    "Clearfelling": "14/770/507/0011/201920/00",
    "Regrowth retention harvesting": "08/286/505/0029/201920/00",
    "Variable retention 1": "16/686/510/0026/201920/00",
}

In [ ]:
# Get a list containing the IDs from the dictionary
event_ids = list(events_of_interest.values())

# Identify the rows that correspond to the IDs in the list
rows_of_interest = logging_season_data.loc[:, "LOGHISTID"].isin(event_ids)

# Make a new table only containing the rows of interest
logging_events_to_analyse = logging_season_data.loc[rows_of_interest, :].reset_index(drop=True)

# View the new table
logging_events_to_analyse

### Exercise: Reviewing the events

Your team is interested in the answers to the following questions:

* **Question 1**: What is the earliest start date across the three events?
* **Question 2**: What is the latest end date across the three events?
* **Question 3**: Which tree species was harvested in all three events?

> **Your task**: Review the table of selected events above, and answer the questions below.

> Double click the text below to add your answers.

### Your answers

**Question 1**: 

**Question 2**:

**Question 3**:

## Loading satellite data for each event

### Connecting to the datacube and constructing the query

In the next cell, your colleauge has provided you with the code to connect to the datacube, and some recommended settings for loading the data. The recommended settings are as follows:

* **time_range**: The date range to search for data over. Your colleague suggests looking at events sometime before and after the actual events.
* **products**: the satellite data to load from. "s2a_ard_granule" is the analysis-ready data product for Sentinel-2A, and "s2b_ard_granule" is the analysis-ready data product for Sentinel-2B.
* **measurements**: the Sentinel-2 bands to load. The "nbart_" prefix has to do with how the data have been processed. For visual interpretation, you need the red, green and blue bands. For calculating NDVI, you need the red and near-infrared (nir_1) bands.
* **resolution**: the resolution (in metres) for each pixel in the image. The first number (-10) specifies 10m in the vertical direction, and the second number (10) specifies 10m in the horizontal direction.
* **output_crs**: the coordinate reference system (CRS) to use for the loaded data. [EPSG:3577](https://epsg.io/3577) is the Australian Albers equal-area projection.
* **min_gooddata**: The proportion of pixels in the image that must be good quality (i.e. not cloudy) for the whole image to be loaded. This makes sure we only load data with minimal cloud coverage. 

In [ ]:
# Connect to the datacube
dc = datacube.Datacube(app="Logging_analysis")

# Recommended settings
time_range = ("2019-06-01", "2020-12-31")
products = ["s2a_ard_granule", "s2b_ard_granule"]
measurements = ["nbart_red", "nbart_green", "nbart_blue", "nbart_nir_1"]
resolution = (10, -10)
output_crs = "EPSG:3577"
min_gooddata = 0.99

### Loading data

The next cell contains multiple steps that are used to load the data for each event. This is done by using a Python `for` loop, which loads data for each event in turn, and stores the outputs.

> **The data loading step will take 5 minutes!** 

Please be patient and watch the output. You can read more about the steps involved as the code runs. The steps are described below the next cell. You will know it is finished when you see the message "All data loading is complete! You can progress to the next step.".

> **When running the code, the following warning may appear**: `CPLReleaseMutex: Error = 1 (Operation not permitted)`. This is normal and you don't need to worry.

Run the code cell below, then scroll down to read about the steps while the data loads.

In [ ]:
# Add the settings into a dictionary, which can be used for all three events.
query = {
    "time": time_range,
    "products": products,
    "measurements": measurements,
    "resolution": resolution,
    "output_crs": output_crs,
    "min_gooddata": min_gooddata,
    "group_by": "solar_day",
}

# Create empty dictionary to store results in
event_data = {}

for (event_type, event_id) in events_of_interest.items():

    print(f"Analysing LOGHISTID {event_id}; Event type: {event_type}")

    # Select the row corresponding to the LOGHISTID value
    event = logging_events_to_analyse.loc[logging_events_to_analyse.LOGHISTID == event_id]

    # Get the polygon geometry for the event and add it to the query
    geometry = Geometry(geom=event.geometry.values[0], crs=logging_events_to_analyse.crs)
    query.update({"geopolygon": geometry})

    # Load the data useing the query
    ds = load_ard(dc=dc, **query)

    # Generate a polygon mask to keep only data within the polygon and apply the mask
    mask = xr_rasterize(event, ds)
    ds = ds.where(mask)

    # Group by 3 month intervals and calculate a composite dataset using the geomedian
    grouped = ds.resample(time="3MS")
    composite = grouped.map(xr_geomedian)

    # Calculate NDVI using (NIR - Red)/(NIR + Red)
    composite["NDVI"] = (composite.nbart_nir_1 - composite.nbart_red) / (composite.nbart_nir_1 + composite.nbart_red)

    # Store the results in the event_data dictionary
    event_data[event_type] = composite
    
    print("\n")
    
print("All data loading is complete! You can progress to the next step.")

### Description of steps for loading data

The Python for loop allows us to repeat the same set of actions for each event. The actions are:

1. **Select the row corresponding to the LOGHISTID value.**

2. **Get the polygon geometry for the event and add it to the query.** This means the datacube will only return data relevent to the event area.

3. **Load the data useing the query.** The `load_ard()` function takes the datacube connection and the query, and loads the relevent data.

4. **Generate a polygon mask to keep only data within the polygon and apply the mask.** This maps the geometry to a raster with ones and zeros, where ones correspond to pixels within the area of interest, and zeros correspond to areas outside the area of interest. This will allow us to isolate out pixels corresponding to the logging event.

5. **Group by 3 month intervals and calculate a composite dataset using the geomedian.** This step allows us to create a representative dataset over a three month period, by selecting the median pixel value from all values loaded for that period. In this case, we use a special type of median, called the geomedian. You can read more about it in the [Digital Earth Africa documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/GeoMAD_specs.html#Geomedian) if you are curious (it is not required to complete the practical).

6. **Calculate NDVI using (NIR - Red)/(NIR + Red)**. This step takes the loaded red and near-infrared bands from the composite and calculates the corresponding NDVI values. NDVI is a satellite band index that indicates the presence of vegetation, with values ranging from -1 to 1. Higher values typically correspond to dense, green vegetation.

7. **Store the results in the event_data dictionary**. This step allows us to store the data for each event and use it for our analysis.

## Visualising loaded data

### Spatial time series
Plotting data in Python can be a little tricky, so your colleague has dug out an old function they once made to help you. The function is called `plot_rgb_ndvi()` and it takes event data and the name of the event type to use as a title. It will show the visual image of the logging area (the combination of red, green and blue bands) on the left-hand side, and the corresponding NDVI values on the right hand side. It will plot each composite that was generated, one after the other.

Run the following cell to view the RGB and NDVI images for each logging event.

In [ ]:
# Plot RGB and NDVI for the event areas
for event_type, event_ds in event_data.items():
    plot_rgb_ndvi(event_ds, event_type)

### Summary time series

Your colleague has provided additional code to calculate and plot the average NDVI value for each composite, which will allow you to see the general trend in the presence of vegetation for the three events.

Run the cell below to view the average NDVI over time.

In [ ]:
# Store the plot labels to add at the end
labels = []

# Create the figure
fig, ax = plt.subplots(figsize=(12, 6))

# Add a title
fig.suptitle("Change in NDVI over time for different logging events", fontsize=16)

# Plot the mean (average) NDVI for each event
for event_type, event_ds in event_data.items():
    event_ds.NDVI.mean(dim=["x", "y"]).plot(add_legend=False, ax=ax)
    labels.append(event_type)

# Add the labels and display the plot
plt.legend(labels, ncol=1, fontsize=12)
plt.show()

## Analysis

Congratulations! You have successfully loaded and visualised satellite data for the three logging events! Now, your colleagues are curious to know what you found.

Your team ask you to report back on the following:

* **Analysis Question 1**: In 1-2 sentences, please describe one similarity you noticed between the three events when looking at the RGB and NDVI spatial time series.
* **Analysis Question 2**: In 1-2 sentences, please describe one difference you noticed between the three events when looking at the RGB and NDVI spatial time series.
* **Analysis Question 3**: When looking at the summary time series for the Clearfelling event, around what month and year would you say the Clearfelling began? Is this consistent with the start date in the event table you created when [selecting logging events for analysis](#Select-logging-events-for-analysis)?

> **Your task**: Review the spatial and summary time series, and answer the analysis questions below.

> Double click the text below to add your answers.

### Your answers

**Analysis Question 1**: 

**Analysis Question 2**:

**Analysis Question 3**:

## Submit your work

1. Ensure you have added answers to all the questions and save your file (in the menu bar, click File > Save Notebook).

2. In the file browser, right-click the `prac2_logging_site_monitoring.ipynb` file, and press "Download"

3. Email the downloaded file to caitlinisabeladams@swin.edu.au